## Here's a script to easily scrape .png's at the desired data coordinateas from the website. (Scroll to bottom, NOTE: this script should be tested more thouroughly to make sure its returning the correct images...)

#### preliminary

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import urllib2

plt.style.use('ggplot')
np.random.seed(1)
url = ('https://raw.githubusercontent.com/Upward-Spiral-Science'
       '/data/master/syn-density/output.csv')
data = urllib2.urlopen(url)
csv = np.genfromtxt(data, delimiter=",")[1:] # don't want first row (labels)
data = csv
sizes = [len(np.unique(data[:, i])) for i in range(3)]
ranges = [(np.max(data[:, i]), np.min(data[:,i])) for i in range(3)]
ranges_diff = [np.max(data[:, i])-np.min(data[:,i]) for i in range(3)]
print sizes
print ranges
print ranges_diff
print np.unique(data[:, 2])[1]-np.unique(data[:, 2])[0]
print np.unique(data[:, 0])[2]-np.unique(data[:, 0])[1]
print np.unique(data[:, 1])[1]-np.unique(data[:, 1])[0]


[108, 52, 11]
[(4192.0, 19.0), (3358.0, 1369.0), (1165.0, 55.0)]
[4173.0, 1989.0, 1110.0]
111.0
39.0
39.0


### Using image data 
#### converting from data units (cx, cy, cz) to pixels 
In the x-direction, we have 108 bins, and in the y-direction, 86 (108/85 ~= 450/350. Furthermore, we have that each xy plane image is 135424 px by 119808 px.

In [2]:
xPix = 135424
yPix = 119808
xPixPerBin = xPix/108.0
yPixPerBin = yPix/86.0
print xPixPerBin, yPixPerBin
# Now since each bin is 40 data coordinates we can define the following function to convert from coordinates to pixels.

1253.92592593 1393.11627907


In [3]:
def coords_to_px(xcoord, ycoord):
    c_vec = np.array([xcoord, ycoord], dtype='float')
    c_vec /= 40.0
    return (c_vec[0]*xPixPerBin, c_vec[1]*yPixPerBin)

# check that max coordinate values are close to max pixels
print coords_to_px(4192, 3358)
# how big is a bin?
print coords_to_px(40, 40)

(131411.43703703702, 116952.11162790698)
(1253.9259259259259, 1393.1162790697674)


#### Grabbing images from website
Looking through the JavaScript code on viz.neurodata.io, we see that its tiling with 512x512 .png images. At resolution 0, each image has 512x512 "pixels" (correspdonding to the brain images) in it. This can be seen b/c total tiles in the x_direction is 264, and 264x512=135168, approximately our max pixels, and for y-direction, max tiles is 233, 233x512=119296, approximately the max pixels. Similarly for resolution 1, each 512x512 pmg now holds 1024x1024 "pixels", and so on as resolution goes up. Also note that at resolution 1, a single tale is very close in size to a single bin.

In [4]:
def get_tilenums_at_res(xcoord, ycoord, res):
    x, y = coords_to_px(xcoord, ycoord)
    x = np.floor(float(x)/(512*(2**res)))
    y = np.floor(float(y)/(512*(2**res)))
    return x,y

def get_image_url(xcoord, ycoord, res, z):
    x, y = get_tilenums_at_res(xcoord, ycoord, res)
    x = int(x)
    y = int(y)
    end = '/'+reduce(lambda x, y: str(x) +'_'+str(y), [y, x, res])
    end += '.png'
    imgurl = 'http://openconnecto.me/ocp/catmaid/bock11/image/xy/'+str(z) +end
    return imgurl

print get_image_url(2000, 2000, 0, 2917)

http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/136_122_0.png


Now just need to figure out z-axis. The z values in the image data go from 2917-4156, which is a range of 

In [5]:
print (2917-4156)*-1

1239


So it seems that the z-values in the data correspond to the z-values in the image data, other than a translation of 2917. So let's redefine our function, and put it in one code block so its easy for other people to use...

In [6]:
xPix = 135424
yPix = 119808
xPixPerBin = xPix/108.0
yPixPerBin = yPix/86.0

def coords_to_px(xcoord, ycoord):
    c_vec = np.array([xcoord, ycoord], dtype='float')
    c_vec /= 40.0
    return (c_vec[0]*xPixPerBin, c_vec[1]*yPixPerBin)

def get_tilenums_at_res(xcoord, ycoord, res):
    x, y = coords_to_px(xcoord, ycoord)
    if(res == 0):
        x = np.floor(float(x)/512)
        y = np.floor(float(y)/512)
    else:
        x = np.floor(float(x)/(512*(2**res)))
        y = np.floor(float(y)/(512*(2**res)))
    return x,y

def get_image_url(xcoord, ycoord, z, res):
    z += 2917
    z = int(z)
    x, y = get_tilenums_at_res(int(xcoord), int(ycoord), res)
    x = int(x)
    y = int(y)
    end = '/'+reduce(lambda x, y: str(x) +'_'+str(y), [y, x, res])
    end += '.png'
    imgurl = 'http://openconnecto.me/ocp/catmaid/bock11/image/xy/'+str(z) +end
    return imgurl

In [7]:
m = np.max(data[:, -1])
a = np.where(data[:, -1]==m)
print data[a]
args = list(*data[a, (0, 1, 2)])+[0]
print args
print get_image_url(*args)

[[   2749.    1876.    1054.  149991.     507.]]
[2749.0, 1876.0, 1054.0, 0]
http://openconnecto.me/ocp/catmaid/bock11/image/xy/3971/127_168_0.png


If everything I did was correct, the above link should be the bin where maximal number of synapses occured.

<img src="maxrel1.png">

In [8]:
args[-1] += 1
print get_image_url(*args)

http://openconnecto.me/ocp/catmaid/bock11/image/xy/3971/63_84_1.png


Change resolution but leave same coordinates to zoom out but leave focal point in center.

<img src="maxrel2.png">